## [Python] Web Scraping - ThreatPost.com
this code is used to scrap the meta-data of articles and their contents via threatpost.com as an example for testing. <br />
<b>Date:</b> 2017/11/20<br />
<b>Loc:</b> III, Taipei, TAIWAN(R.O.C.)<br />
<b>Coder:</b> <i>Chan, TooDou, Chun-Hsiang</i>

## Import packages

In [1]:
# https://threatpost.com/category/cloud-security/
# import packages
import numpy as np
import pandas as pd
import os
import datetime
import sys
from bs4 import BeautifulSoup
import requests
from urllib.request import Request, urlopen

## Obtain all article info from the index.html

In [2]:
# request to open the website via urlopen
req = Request('https://threatpost.com/category/cloud-security/', headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
# html parser via beautifulsoup
soup = BeautifulSoup(webpage, "html.parser")

In [28]:
pageNum = soup.select('.page-numbers')
num_page = len(pageNum)
int(pageNum[num_page-2].text)

20

In [27]:
# choose the news part
full_latest_post = soup.select('#latest-posts')[0]
# count the number of article
num_article = len(full_latest_post.select('article'))
# declare the list for storing the article meta-data
info_categories = []
info_title = []
info_date = []
info_author = []
info_url = []
# parsing the article meta-data to list
for i in range(num_article):
    # get article
    article_contents = full_latest_post.select('article')[i]
    # parsing and storing
    info_categories.append(article_contents.select('.categories')[0].text[14:-1])
    info_title.append(article_contents.select('.entry-title')[0].text[1:-1])
    info_date.append(article_contents.select('.post-info')[0].text[1:-1])
    info_author.append(article_contents.select('.post-info')[1].text[4:-1])
    info_url.append(article_contents.a['href'])
# test [print all out]
# print(info_categories)
# print(info_title)
# print(info_date)
# print(info_author)
# print(info_url)

In [4]:
# save to single dataframe
thretPost_com = pd.DataFrame(np.column_stack([info_categories, info_title, info_date, info_author, info_url])
                             , columns=['categories', 'title', 'date', 'author', 'url'])
# print the head of dataframe
thretPost_com.head()

,categories,title,date,author,url
0,"Cloud Security, Hacks, Privacy",Amazon Promises Fix to Stop Key Service Hack,"November 17, 2017 , 1:59 pm",Tom Spring,https://threatpost.com/amazon-promises-fix-for...
1,"Cloud Security, Privacy, Web Security",Data Pours from Cloud—And ‘The Enemy is Us’,"November 6, 2017 , 8:00 am",Tom Spring,https://threatpost.com/data-pours-from-cloud-a...
2,"Cloud Security, Hacks, Privacy, Vulnerabili...",WordPress Delivers Second Patch For SQL Inject...,"November 1, 2017 , 2:35 pm",Tom Spring,https://threatpost.com/wordpress-delivers-seco...
3,"Cloud Security, Featured, Privacy, Vulnerab...",Slack Plugs ‘Severe’ SAML User Authentication ...,"October 27, 2017 , 8:00 am",Tom Spring,https://threatpost.com/slack-plugs-severe-saml...
4,"Cloud Security, Cryptography, Vulnerabilitie...",Cisco Warns 69 Products Impacted by KRACK,"October 20, 2017 , 7:00 am",Tom Spring,https://threatpost.com/cisco-warns-69-products...


## Obtain all contents of article

In [5]:
# request to open the website via urlopen
req = Request(thretPost_com.url[0], headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
# html parser via beautifulsoup
soupIn = BeautifulSoup(webpage, "html.parser")
# get contents of whole site
all_content = soupIn.select('.entry-content')[0]
# count the number of paragraph
num_paragraph = len(all_content.select('p'))
# cat all paragraph into single string
s = ''
for i in range(num_paragraph):
    t = all_content.select('p')[i].text
    s += ' ' + t

In [6]:
def get_paragraphContents(url):
    # request to open the website via urlopen
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    # html parser via beautifulsoup
    soupIn = BeautifulSoup(webpage, "html.parser")
    # get contents of whole site
    all_content = soupIn.select('.entry-content')[0]
    # count the number of paragraph
    num_paragraph = len(all_content.select('p'))
    # cat all paragraph into single string
    s = ''
    for i in range(num_paragraph):
        if i==0:
            t = all_content.select('p')[i].text
            s = t
        else:
            t = all_content.select('p')[i].text
            s += ' ' + t
    return s

In [7]:
get_paragraphContents(thretPost_com.url[0])

'Researchers at Rhino Security Labs identified a flaw in Amazon’s Key delivery service and Cloud Cam security camera that allows a rogue courier to tamper with the camera and knock it offline, making it appear no one is entering home, when that’s not the case. Amazon Key service allows homeowners to remotely unlock and lock their front door for visitors. The service works in conjunction with Amazon’s Cloud Cam security camera. And if a user is a Prime member, they can permit Amazon couriers authenticate themselves in order to unlock and relock the door to leave a package inside a home on their own. Remote homeowners use the Amazon Key app to monitor their front door via a video feed and receive Amazon delivery alerts. Amazon Prime delivery people also use a version of the Amazon Key app to unlock and lock a customer door. Rhino Labs researchers developed a program that can forge a request from the Wi-Fi router the Cloud Cam device is connected with that tells the camera to stop working